In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from scipy import stats
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [2]:
dataset = pd.read_csv('datasets_9961_14084_Train.csv')
dataset.head()

Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0           FDA15         9.30          Low Fat         0.016047   
1           DRC01         5.92          Regular         0.019278   
2           FDN15        17.50          Low Fat         0.016760   
3           FDX07        19.20          Regular         0.000000   
4           NCD19         8.93          Low Fat         0.000000   

               Item_Type  Item_MRP Outlet_Identifier  \
0                  Dairy  249.8092            OUT049   
1            Soft Drinks   48.2692            OUT018   
2                   Meat  141.6180            OUT049   
3  Fruits and Vegetables  182.0950            OUT010   
4              Household   53.8614            OUT013   

   Outlet_Establishment_Year Outlet_Size Outlet_Location_Type  \
0                       1999      Medium               Tier 1   
1                       2009      Medium               Tier 3   
2                       1999      Medium               Tier 1   
3                       1998         NaN               Tier 3   
4                       1987        High               Tier 3   

         Outlet_Type  Item_Outlet_Sales  
0  Supermarket Type1          3735.1380  
1  Supermarket Type2           443.4228  
2  Supermarket Type1          2097.2700  
3      Grocery Store           732.3800  
4  Supermarket Type1           994.7052

In [3]:
dataset['Item_Identifier'] = dataset['Item_Identifier'].str[0:2]

In [4]:
for i in dataset.columns:
    print((dataset[i].value_counts()/len(dataset))*100)

FD    71.864367
NC    18.761000
DR     9.374633
Name: Item_Identifier, dtype: float64
12.150    1.009034
17.600    0.962103
13.650    0.903438
11.800    0.891705
15.100    0.797841
9.300     0.797841
16.700    0.774375
10.500    0.774375
19.350    0.739176
20.700    0.727443
16.000    0.727443
9.800     0.715710
17.700    0.703977
17.750    0.703977
15.000    0.692245
18.850    0.692245
15.850    0.692245
16.750    0.680512
18.250    0.680512
19.600    0.680512
15.700    0.668779
12.500    0.657046
9.195     0.657046
20.200    0.621847
12.100    0.621847
10.195    0.621847
12.600    0.621847
15.600    0.610114
13.500    0.598381
11.500    0.598381
            ...   
5.305     0.035199
6.305     0.035199
7.710     0.035199
7.105     0.035199
5.825     0.035199
6.460     0.035199
5.425     0.035199
6.895     0.023466
8.920     0.023466
6.325     0.023466
6.400     0.023466
4.905     0.023466
5.675     0.023466
5.155     0.023466
7.275     0.023466
5.210     0.023466
8.485     0.023466
8.

In [5]:
dataset.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [6]:
dataset['Item_Weight'].fillna(dataset['Item_Weight'].mean(), inplace = True)

In [7]:
dataset['Outlet_Size'].fillna(dataset['Outlet_Size'].mode()[0], inplace = True)

In [8]:
dataset.pivot_table(index= 'Outlet_Size',values = 'Item_Outlet_Sales')

Item_Outlet_Sales
Outlet_Size                   
High               2298.995256
Medium             2283.730470
Small              1912.149161

In [9]:
dataset['Age_Of_Company'] = 2020 - dataset['Outlet_Establishment_Year']
dataset['Age_Of_Company'].describe()

count    8523.000000
mean       22.168133
std         8.371760
min        11.000000
25%        16.000000
50%        21.000000
75%        33.000000
max        35.000000
Name: Age_Of_Company, dtype: float64

In [10]:
dataset['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [11]:
for i in dataset.columns:
    print(dataset[i].unique())

['FD' 'DR' 'NC']
[ 9.3         5.92       17.5        19.2         8.93       10.395
 13.65       12.85764518 16.2        11.8        18.5        15.1
 17.6        16.35        9.         13.35       18.85       14.6
 13.85       13.          7.645      11.65        5.925      19.25
 18.6        18.7        17.85       10.          8.85        9.8
 13.6        21.35       12.15        6.42       19.6        15.85
  7.39       10.195       9.895      10.895       7.905       9.195
  8.365       7.97       17.7        19.35        8.645      15.6
 18.25        7.855       7.825       8.39       12.85       19.
  5.905       7.76       16.75       15.5         6.055       6.305
 20.85       20.75        8.895      19.7         8.75       13.3
  8.31       19.75       17.1        10.5         6.635      14.15
  8.89        9.1         7.5        16.85        7.485      11.6
 12.65       20.25        8.6        12.6         8.88       20.5
 13.5         7.235       6.92        8.02       12

In [12]:
dataset['Item_Fat_Content'] = dataset['Item_Fat_Content'].replace({'Low Fat':0, 'Regular':1, 'low fat':0, 'LF':0, 'reg':1})

In [13]:
#dataset['Item_Identifier'] = dataset['Item_Identifier'].replace({'FD':0, 'DR':1, 'NC':2})

In [14]:
dataset['Outlet_Size'] = dataset['Outlet_Size'].replace({'Small':0,'Medium':1,'High':2})

In [15]:
dataset['Outlet_Location_Type'] = dataset['Outlet_Location_Type'].replace({'Tier 1':0,'Tier 2':1,'Tier 3':2})

In [16]:
dataset['Outlet_Type'] = dataset['Outlet_Type'].replace({'Supermarket Type2':0,'Supermarket Type3':1,'Grocery Store':2,'Supermarket Type1':3})

In [17]:
dataset.head()

Item_Identifier  Item_Weight  Item_Fat_Content  Item_Visibility  \
0              FD         9.30                 0         0.016047   
1              DR         5.92                 1         0.019278   
2              FD        17.50                 0         0.016760   
3              FD        19.20                 1         0.000000   
4              NC         8.93                 0         0.000000   

               Item_Type  Item_MRP Outlet_Identifier  \
0                  Dairy  249.8092            OUT049   
1            Soft Drinks   48.2692            OUT018   
2                   Meat  141.6180            OUT049   
3  Fruits and Vegetables  182.0950            OUT010   
4              Household   53.8614            OUT013   

   Outlet_Establishment_Year  Outlet_Size  Outlet_Location_Type  Outlet_Type  \
0                       1999            1                     0            3   
1                       2009            1                     2            0   
2                       1999            1                     0            3   
3                       1998            1                     2            2   
4                       1987            2                     2            3   

   Item_Outlet_Sales  Age_Of_Company  
0          3735.1380              21  
1           443.4228              11  
2          2097.2700              21  
3           732.3800              22  
4           994.7052              33

In [20]:
from sklearn.linear_model import LinearRegression
X= dataset[['Age_Of_Company','Item_MRP','Item_Fat_Content','Outlet_Size','Outlet_Location_Type']]
Y = dataset['Item_Outlet_Sales']


In [21]:
reg = LinearRegression().fit(X,Y)

In [22]:
reg.score(X,Y)

0.3336572235747527

In [23]:
dataset = pd.get_dummies(dataset, drop_first=True)
dataset.head()

Item_Weight  Item_Fat_Content  Item_Visibility  Item_MRP  \
0         9.30                 0         0.016047  249.8092   
1         5.92                 1         0.019278   48.2692   
2        17.50                 0         0.016760  141.6180   
3        19.20                 1         0.000000  182.0950   
4         8.93                 0         0.000000   53.8614   

   Outlet_Establishment_Year  Outlet_Size  Outlet_Location_Type  Outlet_Type  \
0                       1999            1                     0            3   
1                       2009            1                     2            0   
2                       1999            1                     0            3   
3                       1998            1                     2            2   
4                       1987            2                     2            3   

   Item_Outlet_Sales  Age_Of_Company  ...  Item_Type_Starchy Foods  \
0          3735.1380              21  ...                        0   
1           443.4228              11  ...                        0   
2          2097.2700              21  ...                        0   
3           732.3800              22  ...                        0   
4           994.7052              33  ...                        0   

   Outlet_Identifier_OUT013  Outlet_Identifier_OUT017  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         1                         0   

   Outlet_Identifier_OUT018  Outlet_Identifier_OUT019  \
0                         0                         0   
1                         1                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   Outlet_Identifier_OUT027  Outlet_Identifier_OUT035  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   Outlet_Identifier_OUT045  Outlet_Identifier_OUT046  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   Outlet_Identifier_OUT049  
0                         1  
1                         0  
2                         1  
3                         0  
4                         0  

[5 rows x 36 columns]

In [24]:
x,y =dataset.drop(['Item_Outlet_Sales'],axis =1),dataset.Item_Outlet_Sales

In [25]:
x.shape,y.shape

((8523, 35), (8523,))

In [26]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 100)

In [27]:
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [28]:
lr_pred = lr.predict(x_test)
from sklearn.metrics import r2_score
r2_score(y_test,lr_pred)

0.5496568184359295